In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

base_dir = "/home/up2212278/Project/Normal_vs_Military_Vehicles/"

train_dir = f"{base_dir}train"
val_dir = f"{base_dir}validation"
test_dir = f"{base_dir}test"

def get_data_generators(train_dir, val_dir, test_dir, image_size=224, batch_size=24):
    data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = data_generator.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical'
    )

    validation_generator = data_generator.flow_from_directory(
        val_dir,
        target_size=(image_size, image_size),
        class_mode='categorical'
    )

    test_generator = data_generator.flow_from_directory(
        test_dir,
        target_size=(image_size, image_size),
        class_mode='categorical'
    )

    return train_generator, validation_generator, test_generator

train_generator, validation_generator, test_generator = get_data_generators(train_dir, val_dir, test_dir)

def read_and_prep_images(img_paths, img_height=224, img_width=224):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return output

model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(2, activation='softmax'))
model.layers[0].trainable = False

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

start_time = time.time()

model.fit(
    train_generator,
    steps_per_epoch=466,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=146
)


end_time = time.time()

training_time_seconds = end_time - start_time
training_time_minutes = training_time_seconds / 60

print(f"Total training time: {training_time_seconds:.2f} seconds ({training_time_minutes:.2f} minutes)")

results = model.evaluate(test_generator)
print(f"Test Accuracy: {results[1]:.4f}")

model.save("Millitary_vers_01_model.h5")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model
import requests
from io import BytesIO
from PIL import Image

def classify_image_from_url(img_url, model):
    response = requests.get(img_url)
    img_data = BytesIO(response.content)
    img = Image.open(img_data)

    img = img.resize((224, 224))
    preprocessed_img = preprocess_input(np.expand_dims(img_to_array(img), axis=0))

    preds = model.predict(preprocessed_img)
    class_labels = ["millitary", "other"]
    predicted_class = class_labels[np.argmax(preds[0])]

    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Predicted Class: {predicted_class}\n{preds}")
    plt.axis('off')
    plt.show()

image_urls = [
    "https://mvs.gov.ua/upload/1/7/9/2/6/7/mceu-51095692011707836227352.jpg",
    "https://im.kommersant.ru/Issues.photo/CORP/2022/02/24/KMO_141501_08820_1_t222_151046.jpg",
    "https://mvs.gov.ua/upload/1/7/9/2/6/8/mceu-83735056321707836253435.jpg",
    "https://mvs.gov.ua/upload/1/7/9/2/7/2/mceu-78158512061707836612027.jpg",
    "https://gdb.rferl.org/049b0000-0aff-0242-7385-08da1613f320_w408_r1_s.jpg",
    "https://mvs.gov.ua/upload/1/7/9/2/8/3/mceu-33443764741707838555964.jpg",
    "https://www.rbc.ua/static/img/_/v/_vnochi_foto_vidkriti_dzherela_jpg1_3bdd69c775f07b31c72ed092b6fde0ef_1200x675.jpg"
]

model_01 = load_model("Millitary_vers_01_model.h5")

for url in image_urls:
    classify_image_from_url(url, model_01)
model_01 = load_model("Millitary_vers_01_model.h5")

for url in image_urls:
    classify_image_from_url(url, model_01)


Number of correct predictions - 5/7
Accuracy - 0.714%